# main ideas

## The first stock Pattern I want to find is:

* we need the time to be hours, the first 12-24 hours

0. the 5-10% doesn't mean the stock price end on that day with a 5% raise, it means during the 1-2 days, it reach that point. so stage1 and stage2 also doesn't have to be in day 1 stage1, day 2-5 stage2, it can be day1 stage1, day1 also starts stage2, the day1 2 3 4 is more about the length.
1. Stage1, Big rasie: the first 6-12 hours(2 days), the stock raise about 5-10%, 

2. Stage2: Consolidating: after that rise, the 6-30 hours(2-6 days) days the stock consolidating around that price(no more than 5% than stage1 lowest or highest price )   
3. Stage3: Decreasing Volume: it shows Doji/or other indecisive trading siganl with decreasing volume right after at least 12 hours min 30 hours max 2 days(at most 5days) of consolidating

will it make difference though? 

## Buy Point: and we buy at the next day of stage3 siganl
## Sell Point: we sell at the next day of buying 

I want to code to be:
0.modular so that I can adjust and add other stage and easy to maintain
1.accepet parameters(like stage1 it may be 7-12% instead of 5-10%, the last period of stage 1 maybe 1-4days instead of 1-2days)
2.I can test it with simulation, like I run it during a period of time form 2025/1/1 to 2025/3/1 and buy and the mid price at the buy point, and sell at the mid price at the sell point, how many profits can I have
3.I can use the simulation resualts to adjust the parameters to get more profit automatically 




## Can we use code to detect the patterns?
* we observe and research the profitable patterns, we observe instead of guessing from no reason,
    * for the consolidating time
        * at least up down and up 
        * or just stay there 


* use code to optimize the profit
* first we can use code to detect the hot rasing areas like IT / medical / 
* classify the rasing stocks based on different 
    * market times 


In [1]:
import yfinance as yf
import pandas as pd
code = 'AAPL'
start_date = '2025-05-07'
end_date ='2025-05-09'
df = yf.download("AAPL", start=start_date, end=end_date, interval="1d")
print(df)

ModuleNotFoundError: No module named 'yfinance'

In [1]:
# pattern.py

import pandas as pd
from dataclasses import dataclass

# We define a PatternParams class to hold all tunable parameters for the pattern definition. 
# This makes it easy to adjust thresholds or extend with new parameters. 

# is_stage3_signal(idx, daily_df): Checks if the day at idx in the daily DataFrame is an indecisive candlestick (doji) with volume lower 
# than previous days (Stage 3 criteria).

# is_consolidation(start_idx, end_idx, daily_df): Checks if the price between start_idx and end_idx (inclusive) stayed within a tight range 
# (Stage 2 criteria).

# is_big_rise(start_idx, end_idx, daily_df): Checks if there was a rise of the required percentage between start_idx and end_idx (Stage 1 criteria).

# find_signals(hourly_df): Converts hourly data to daily, then iterates through each potential Stage 3 day and verifies if preceding Stage 2 and Stage 1 
# conditions are met. Returns a list of detected pattern signals (with buy/sell info).

# pattern_hourly.py

import pandas as pd
from dataclasses import dataclass

@dataclass
class PatternParams:
    # Stage 1: Big Rise
    rise_min: float = 0.02           # 5%
    rise_max: float = 0.10           # 10%
    stage1_min_hours: int = 6
    stage1_max_hours: int = 12

    # Stage 2: Consolidation
    cons_min_hours: int = 6
    cons_max_hours: int = 30
    cons_max_range: float = 0.09     # 5% range

    # Stage 3: Doji + Volume Drop
    doji_body_frac: float = 0.6
    vol_compare_hours: int = 2       # must be lower than N previous bars

class PatternDetector:
    def __init__(self, params: PatternParams = None):
        self.params = params or PatternParams()

    def is_doji(self, row) -> bool:
        high, low = row['High'], row['Low']
        open_, close = row['Open'], row['Close']
        if high == low:
            return False
        body = abs(open_ - close)
        range_ = high - low
        return body <= self.params.doji_body_frac * range_

    def is_volume_dropped(self, idx: int, df: pd.DataFrame) -> bool:
        if idx < self.params.vol_compare_hours:
            return False
        curr_vol = df.iloc[idx]['Volume']
        prev_vol = df.iloc[idx - self.params.vol_compare_hours:idx]['Volume']
        return (prev_vol > curr_vol).all()

    def is_consolidating(self, df: pd.DataFrame) -> bool:
        high = df['High'].max()
        low = df['Low'].min()
        return (high - low) / low <= self.params.cons_max_range

    def find_signals(self, df: pd.DataFrame):
        signals = []
        n = len(df)
        for i in range(n - self.params.stage1_min_hours - self.params.cons_min_hours - 2):
            # Stage 1: Look for rise
            for h1 in range(self.params.stage1_min_hours, self.params.stage1_max_hours + 1):
                stage1 = df.iloc[i:i + h1]
                if stage1.empty:
                    continue
                low = stage1['Low'].min()
                high = stage1['High'].max()

                try:
                    low = float(low.iloc[0]) if hasattr(low, 'iloc') else float(low)
                    high = float(high.iloc[0]) if hasattr(high, 'iloc') else float(high)
                except:
                    continue

                if low == 0:
                    continue

                rise = (high - low) / low

                try:
                    rise = float(rise)
                except:
                    continue

                if not (self.params.rise_min <= rise <= self.params.rise_max):
                    continue

                # Stage 2: Look for consolidation right after Stage 1
                stage2_start = i + h1
                for h2 in range(self.params.cons_min_hours, self.params.cons_max_hours + 1):
                    if stage2_start + h2 + 1 >= n:
                        break
                    stage2 = df.iloc[stage2_start:stage2_start + h2]
                    if stage2.empty:
                        continue
                    try:
                        if not bool(self.is_consolidating(stage2)):
                            continue
                    except Exception:
                        continue

                    # Stage 3: Doji + low volume
                    stage3_idx = stage2_start + h2
                    row = df.iloc[stage3_idx]
                    if self.is_doji(row) and self.is_volume_dropped(stage3_idx, df):
                        # Buy next bar, sell one bar later
                        if stage3_idx + 2 < n:
                            buy_row = df.iloc[stage3_idx + 1]
                            sell_row = df.iloc[stage3_idx + 2]
                            trade = {
                                "stage1_range": rise,
                                "buy_time": buy_row.name,
                                "sell_time": sell_row.name,
                                "buy_price": float(buy_row['Open']),
                                "sell_price": float(sell_row['Open']),
                                "profit": float(sell_row['Open'] - buy_row['Open'])
                            }
                            signals.append(trade)
                    break  # break inner loop after valid stage2 attempt
                break  # break if stage1 found
        return signals


In [2]:
# backtest_hourly.py

import pandas as pd

def run_backtest(signals: list) -> pd.DataFrame:
    """
    Run a backtest on a list of trade signals.
    Each signal should have buy_time, sell_time, buy_price, sell_price, and profit.
    """
    if not signals:
        return pd.DataFrame(columns=[
            "buy_time", "sell_time", "buy_price", "sell_price", "profit"
        ])

    df = pd.DataFrame(signals)
    df["return_pct"] = (df["sell_price"] - df["buy_price"]) / df["buy_price"] * 100
    df["cumulative_profit"] = df["profit"].cumsum()
    df["cumulative_return"] = df["return_pct"].cumsum()
    return df



In [3]:
# data.py
# We use the yfinance library to fetch hourly stock data from Yahoo Finance. 
# The fetch_data function retrieves the hourly OHLCV (Open, High, Low, Close, Volume) data for a given stock symbol and date range. 
# We also include a get_stock_metadata function to fetch basic company info (like sector and industry), 
# which could be used in extended logic (though our pattern doesn’t utilize it yet).

import yfinance as yf
import pandas as pd

def fetch_data(symbol: str, start_date: str, end_date: str, interval: str = "1h") -> pd.DataFrame:
    df = yf.download(symbol, start=start_date, end=end_date, interval=interval, auto_adjust=False, progress=False)
    if df.empty:
        raise ValueError(f"No data returned for {symbol}. Check ticker or network.")
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC").tz_convert("America/New_York")
    else:
        df.index = df.index.tz_convert("America/New_York")

    return df


def get_stock_metadata(symbol: str) -> dict:
    """
    Retrieves basic metadata for the given stock (e.g., sector, industry).
    :param symbol: Stock ticker symbol.
    :return: Dictionary with metadata like {'sector': ..., 'industry': ...}.
    """
    ticker = yf.Ticker(symbol)
    info = ticker.info
    return {
        "sector": info.get("sector"),
        "industry": info.get("industry"),
        "long_name": info.get("longName"),
    }


In [4]:
# run_hourly_pattern.py

from pattern import PatternDetector, PatternParams
from backtest import run_backtest
from data import fetch_data
import sys
from yahoo_fin import stock_info as si
import pandas as pd
import yfinance as yf

# df1 = pd.DataFrame(si.tickers_sp500())
# df1 = pd.DataFrame( si.tickers_nasdaq() )
df1 = pd.DataFrame( si.tickers_dow() )
symbols = df1[0].tolist()[:50]
print(len(df1))

start_date = "2024-03-01"
end_date = "2025-03-01"


for symbol in symbols:
    print(f"\n=== Running for {symbol} ===")
    try:
        df = fetch_data(symbol, start_date, end_date, interval="1h")
        detector = PatternDetector(PatternParams())
        signals = detector.find_signals(df)
        results = run_backtest(signals)

        print("\n--- Trade Summary ---")
        print(results)
        print("\nTotal Trades:", len(results))
        print("Total Profit:", results['profit'].sum() if 'profit' in results else 0)
        if 'return_pct' in results.columns:
            print("Total Return %:", results['return_pct'].sum())
        else:
            print("Total Return %: 0")

    except Exception as e:
        print(f"Error processing {symbol}: {e}")


Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.
30

=== Running for AAPL ===



1 Failed download:
['AAPL']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


Error processing AAPL: No data returned for AAPL. Check ticker or network.

=== Running for AMGN ===



1 Failed download:
['AMGN']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


Error processing AMGN: No data returned for AMGN. Check ticker or network.

=== Running for AMZN ===


...

Error processing V: No data returned for V. Check ticker or network.

=== Running for VZ ===



1 Failed download:
['VZ']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


Error processing VZ: No data returned for VZ. Check ticker or network.

=== Running for WMT ===



1 Failed download:
['WMT']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


Error processing WMT: No data returned for WMT. Check ticker or network.


In [1]:
import requests
import pandas as pd

# 🔐 Replace this with your actual Alpha Vantage API key
ALPHA_VANTAGE_API_KEY = "W5EXBDLHMG8TRB7M"
BASE_URL = "https://www.alphavantage.co/query"

def fetch_data(symbol, interval="60min"):
    """
    Fetches intraday stock data (60-minute intervals) from Alpha Vantage.
    
    Args:
        symbol (str): Stock ticker (e.g., "AAPL")
        interval (str): Time interval ("60min" recommended)

    Returns:
        pd.DataFrame: DataFrame with datetime index and OHLCV columns
    """
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": interval,
        "apikey": ALPHA_VANTAGE_API_KEY,
        "outputsize": "full",
        "datatype": "json"
    }

    print(f"Requesting Alpha Vantage data for {symbol}...")
    response = requests.get(BASE_URL, params=params)

    if response.status_code != 200:
        raise RuntimeError(f"Request failed: {response.status_code} — {response.text}")

    data = response.json()
    key = f"Time Series ({interval})"
    if key not in data:
        raise ValueError(f"Unexpected response format: {data}")

    df = pd.DataFrame.from_dict(data[key], orient="index")
    df.columns = ["Open", "High", "Low", "Close", "Volume"]
    df = df.astype(float)
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)
    df.index = df.index.tz_localize("America/New_York")

    return df[["Open", "High", "Low", "Close", "Volume"]]


In [2]:
print(fetch_data("AAPL"))

Requesting Alpha Vantage data for AAPL...
                             Open      High       Low     Close      Volume
2025-04-09 04:00:00-04:00  172.10  177.1900  172.1000  175.5400    509199.0
2025-04-09 05:00:00-04:00  175.54  176.4500  174.4000  175.1900    212987.0
2025-04-09 06:00:00-04:00  175.17  177.1400  173.8500  176.1600    303209.0
2025-04-09 07:00:00-04:00  176.09  176.4600  168.3800  168.7400   1369332.0
2025-04-09 08:00:00-04:00  174.96  188.4220  168.0000  172.0100   2455832.0
...                           ...       ...       ...       ...         ...
2025-05-08 15:00:00-04:00  198.62  199.6880  197.1640  197.4000   7354331.0
2025-05-08 16:00:00-04:00  197.49  202.0051  197.2000  197.3000  18970508.0
2025-05-08 17:00:00-04:00  197.30  202.0051  197.1500  197.5000   1246081.0
2025-05-08 18:00:00-04:00  197.50  197.5000  197.3301  197.4994     74501.0
2025-05-08 19:00:00-04:00  197.49  197.6000  197.3500  197.4400     40057.0

[336 rows x 5 columns]


In [13]:
print("Python executable:", sys.executable)

Python executable: c:\Users\22348\Capital2025\Stocks\.venv\Scripts\python.exe
